----------
# Write-Up Final Submission - Lukas Burgholzer

### Problem Description
In this challenge, we want to solve a version of the so-called "False Asteroids" puzzle. To this end, we are given a total of sixteen $4\times4$ boards, each with exactly six asteroids placed in certain locations.    
Solving a single board means to shoot a sequence of horizontal and/or vertical beams so that all asteroids are hit by at least one beam.  
We are promissed that fifteen out of the sixteen boards can be solved within three beams, while the remaining board can only be solved using four beams.  
The goal of this challenge is to use Grover's algorithm to find this "special" board.  

Before thinking about the quantum circuit for solving this task, we need to ask ourselves: "How can this board be identified?"

### Classical Considerations
The problem whether a given board cannot be solved by less than four beams readily translates to a minimum vertext cover problem (or similarly a perfect matching problem) on a bipartite graph, where the graph's
- partitions are given by $\{H_0, H_1, H_2, H_3\}$ and $\{V_0, V_1, V_2, V_3\}$, corresponding to shooting a beam horizontally/vertically at the row/column given by the node's index.
- edges are given by the asteroids coordinates, i.e. an asteroid at position $(i,j)$ implies that there is an edge between $H_i$ and $V_j$.

There are multiple ways of tackling this problem. The following statement (see page 4 of [this pdf](https://courses.grainger.illinois.edu/cs598cci/sp2020/LectureNotes/Matchings-via-determinants-scribed.pdf)) provides some guidance:  
> *A bipartite graph has a perfect matching (i.e., is only solveable by 4 beams) whenever the determinant of the **symbolic** matrix corresponding to the graph (i.e., the board matrix) is non-zero **as a polynomial**.*

In general, the calculation of the determinant of a $4\times 4$ matrix involves the calculation and addition/subtraction of 24 conjunctive terms (each consisting of 4 variables).Namely, 
$$
         +(afkp) -(aflo) -(agjp) +(agln) +(ahjo) -(ahkn)\\    
         -(bekp) +(belo) +(bgip) -(bglm) -(bhio) +(bhkm)\\ 
         +(cejp) -(celn) -(cfip) +(cflm) +(chin) -(chjm)\\  
         -(dejo) +(dekn) +(dfio) -(dfkm) -(dgin) +(dgjm)\\
$$
given that the matrix has elements
$$
 a b c d\\
 e f g h\\
 i j k l\\
 m n o p\\
$$
The statement from above is easyly misinterpreted as:  
> *A given board is only solveable with four beams $\leftrightarrow \det(Board) \neq 0$, where $Board[i][j] = 1$ <-> there is an asteroid in position $(i,j)$.*

However, there are instances of boards with a determinant equal to zero, which are still only solvable with four beams.
The key point in the statement above is that the matrix has to be considered as a *symbolic* matrix (not as a binary matrix).
Consequently, whenever at least one of the terms in the calculation of the determinant is not equal to the zero polynomial, there exists a maximal matching.  
In the challenge we only have binary board matrices and no way create a symbolic matrix in the quantum ciruit.
Thus, just calculating the determinant is not enough. See also [this post on StackOverflow](https://math.stackexchange.com/questions/2228262/the-determinant-and-perfect-matching).   
However, there exists a quantity closely related to the determinant, that does the trick; namely the *permanent*.
The permanent of a matrix is just the sum of all the individual terms that would normaly contribute to the determinant of the matrix (i.e., disregarding the minus).
Now, it holds that
> *A given board is only solveable with four beams $\leftrightarrow perm(Board) \gt 0$, where $Board[i][j] = 1$ <-> there is an asteroid in position $(i,j)$.*

This gives us actionable advice for what to calculate. Moreover, some knowledge about the problem itself can be exploited to reduce the amount of computations necessary.     
It is apriori known, that the board is $4\times 4$ and contains exactly six ones. 
One can show (the arguments are related to [Hadamard's maximum determinant problem](https://mathworld.wolfram.com/HadamardsMaximumDeterminantProblem.html)) that this implies that the permanent of the matrix is either $0$, $1$, or $2$.
This means, that in case the board is not the board we are looking for, none of the 24 conditions trigger (which corresponds to the determinant of the matrix being the zero polynomial), while, in case of the target board, up to 2 conditions trigger.

### Design of the Oracle
The goal of the oracle part in Grover's algorithm is to mark the desired target state. This is typically done by flipping the phase of a flag qubit.  
In principle, my oracle just computes the permanent of the board matrix and adds a corresponding phase to a flag qubit whenever the permanent is non-zero.  
One general way of doing this would be to take two auxillary qubits as a counter, which is incremented whenever one of the 24 individual terms evaluates to $1$, and then, afterwards, check whether the count is non-zero.

However, the theoretical considerations from above have shown, that only a maximum of two terms trigger during the permanent calculation.  
Consequently, the additions (i.e. the counting) in the general formula can be replaced by just adding a phase of $\frac{3\pi}{4}$ to the flag qubit whenever a condition triggers.
In case only one condition triggers (which actually happens most of the time) a phase of $\frac{3\pi}{4}$ is applied, while a total phase of $\frac{6\pi}{4}\equiv -\frac{\pi}{2}$ is applied in case two conditions trigger.  
On the positive side, this avoids all the counting, makes a lot of room for using the available qubits to their full extent and requires almost no uncomputation.  
On the negative side, the phase difference between the two cases also implies that the corresponding solutions are amplified to a different extent.  
For one iteration of Grover's algorithm, the achieved probabilities for the target state are $\approx42\%$ in case $perm(Board)=1$, and $\approx27\%$ in case $perm(Board)=2$, whereas non-target states have a probability of $\approx 4-5\%$.

### Loading the Data

The boards' data is loaded using a QRAM structure with 4 address qubits, 16 data qubits and a total of 3 ancillary qubits. The order of 'writes' is a 4-bit [Gray code](https://en.wikipedia.org/wiki/Gray_code) sequence starting from $1111$, i.e.,  
$$
    1111 \rightarrow 1110 \rightarrow 1010\rightarrow 1011 \rightarrow
    1001\rightarrow 1000\rightarrow 0000\rightarrow 0001\rightarrow
    0011\rightarrow 0010\rightarrow 0110\rightarrow 0111\rightarrow
    0101\rightarrow 0100\rightarrow 1100\rightarrow 1101
$$
This way, only one bit changes between subsequent writes, which minimizes the (un-)computations that is necessary between different addresses.

### The Whole Circuit
The complete circuit is then composed of
  - Initialization
  - QRAM
  - Oracle
  - QRAM$^{-1}$
  - Diffusion

It uses 4 address qubits, 16 data qubits, a flag qubit and 7 ancillary qubits.

### Optimizations
The above considerations allow to reliably solve the problem at hand using Grover's algorithm. In order to reduce the cost of the resulting circuit, several optimizations have been applied. The following list aims to provide an overview:
- All (multi-controlled) Toffoli gates uses the maximum of available ancillaries (which involved writing these decompositions by hand) in order to store temporary results and save gates.
- All Toffoli gates which are uncomputed later in the circuit can be realized using relative-phase Toffoli gates (*rccx*). This includes all of the QRAM and large parts of the oracle and the diffusion.
- Observant ordering of computations gives rise to many situations where whole Toffoli gates can be cancelled.
- Two Toffoli gates acting on the same target line and sharing a control can be implemented using fewer CNOT gates than two individual Toffoli gates (this also holds for relative-phase Toffoli gates)
- Implementing a controlled-phase gate is more efficient than implementing a Toffoli gate
- Careful inspection of the circuit allows to cancel many consecutive single-qubit gates throughout the whole circuit.
- IBM devices support arbitrary single-qubit gates. Thus, every sequence of consecutive single-qubit gates may be fused into a single gate.

### Final Cost
Overall, my design resulted in a total cost of **6552** using *612 CX* and *432 U3 gates*.

In [1]:
problem_set = \
    [[['0', '2'], ['1', '0'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '3']],
    [['0', '0'], ['1', '1'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
    [['0', '2'], ['1', '0'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '1'], ['3', '3']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '3'], ['1', '2'], ['2', '2'], ['2', '3'], ['3', '0']],
    [['0', '3'], ['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '3'], ['2', '1'], ['2', '3'], ['3', '0']],
    [['0', '1'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '2']],
    [['0', '0'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '3'], ['3', '1']],
    [['0', '1'], ['0', '2'], ['1', '0'], ['1', '2'], ['2', '2'], ['2', '3']],
    [['0', '3'], ['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '0']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '3'], ['3', '0'], ['3', '1']],
    [['0', '1'], ['1', '0'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '1']]]

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.qasm import pi

""" 
    Given a pair ['i', 'j'] returns the corresponding (1D) index on the board
"""
def idx(meteor):
    return int(meteor[0]) * 4 + int(meteor[1])


""" 
    Optimized version of the RCCX gate of Qiskit 
    fused single qubit gates together and additionally allows to exclude the first or last single qubit gate    
"""
def rccx_opt(side = 'both'):
    a = QuantumRegister(1)
    b = QuantumRegister(1)
    target = QuantumRegister(1)
    qc = QuantumCircuit(a, b, target)

    if side in ['both', 'left']:
        qc.u(pi/2, pi/4, pi, target)       # -H-T-
    qc.cx(b, target)
    qc.tdg(target)
    qc.cx(a, target)
    qc.t(target)
    qc.cx(b, target)
    if side in ['both', 'right']:
        qc.u(pi/2, 0, 3*pi/4, target)    # -Tdg-H-
    return qc.to_gate(label='rccx')



""" 
    Optimized version of the CCX gate of Qiskit 
    fused single qubit gates together and additionally allows to exclude the first or last single qubit gate    
"""
def ccx_opt(side='both'):
    a = QuantumRegister(1)
    b = QuantumRegister(1)
    target = QuantumRegister(1)
    qc = QuantumCircuit(a, b, target)
    if side in ['both', 'left']:
        qc.h(target)
    qc.cx(b, target)
    qc.tdg(target)
    qc.cx(a, target)
    qc.t(target)
    qc.cx(b, target)
    qc.tdg(target)
    qc.cx(a, target)
    qc.t(b)
    qc.cx(a, b)
    qc.t(a)
    qc.tdg(b)
    qc.cx(a, b)
    if side in ['both', 'right']:
        qc.u(pi/2, 0, 5*pi/4, target)  # -T-H-
    return qc.to_gate(label='ccx')


"""
    optimized version of two Toffoli gates targeted at the same qubit and sharing a control
    10 CNOTs + 10 single qubit gates --> 110 cost
""" 
def double_ccx_opt(side='both'):
    a = QuantumRegister(1)
    b = QuantumRegister(1)  # shared control
    c = QuantumRegister(1)
    target = QuantumRegister(1)
    qc = QuantumCircuit(a, b, c, target)

    if side in ['both', 'left']:
        qc.u(pi/2, -pi/4, pi, target)
    qc.cx(a, b)
    qc.tdg(a)
    qc.t(b)
    qc.cx(a, b)
    qc.cx(a, target)
    qc.t(target)
    qc.cx(b, target)
    qc.tdg(target)
    qc.cx(a, target)
    qc.cx(c, target)
    qc.t(target)
    qc.cx(b, target)
    qc.tdg(target)
    qc.cx(c, target)
    qc.cx(c, b)
    qc.tdg(b)
    qc.t(c)
    qc.cx(c, b)
    if side in ['both', 'right']:
        qc.u(pi/2, 0, 5*pi/4, target)

    return qc.to_gate(label='2ccx')


"""
    optimized version of two double-controlled (3*pi/4)-phase-gates targeted at the same qubit and sharing a control
    8 CNOTs + 8 single qubit gates --> 88 cost
""" 
def double_ccp_opt():
    a = QuantumRegister(1)
    b = QuantumRegister(1)
    c = QuantumRegister(1)
    target = QuantumRegister(1)
    qc = QuantumCircuit(a, b, c, target)
    qc.cx(a, target)
    qc.p(-3*pi/8, target)
    qc.cx(b, target)
    qc.p(3*pi/8, target)
    qc.cx(a, target)
    qc.p(-3*pi/8, target)
    qc.cx(b, target)
    qc.p(3*pi/8, target)
    qc.cx(b, target)
    qc.p(-3*pi/8, target)
    qc.cx(c, target)
    qc.p(3*pi/8, target)
    qc.cx(b, target)
    qc.p(-3*pi/8, target)
    qc.cx(c, target)
    qc.p(3*pi/8, target)
    return qc.to_gate(label='2ccp')


"""
    Optimized version of two relative-phase Toffoli gates targeted at the same qubit and sharing a control
    4 CNOTs + 4 single qubit gates --> 44 cost
"""
def double_rccx_opt(side ='both'):
    a = QuantumRegister(1)
    b = QuantumRegister(1)  # shared control
    c = QuantumRegister(1)
    target = QuantumRegister(1)
    qc = QuantumCircuit(a, b, c, target)

    if side in ['both', 'left']:
        qc.u(pi/2, pi/4, pi, target)
    qc.cx(b, target)
    qc.tdg(target)
    qc.cx(a, target)
    qc.cx(c, target)
    qc.t(target)
    qc.cx(b, target)
    if side in ['both', 'right']:
        qc.u(pi/2, 0, 3*pi/4, target)

    return qc.to_gate(label='2rccx')


"""
    QRAM implementation using 4 address (qu)bits, 16 data (qu)bits and a total of 3 ancillary qubits.
    The order of 'writes' is a 4-bit Gray code sequence starting from '1111'. 
    This minimizes the (un-)computations between different addresses.
"""
def qram(problem_set):
    address = QuantumRegister(4)  # address of qram
    data = QuantumRegister(16)  # data of qram
    anc = QuantumRegister(3)  # ancillary
    qc = QuantumCircuit(address, data, anc)

    graycode = ['1111', '1110', '1010', '1011',
                '1001', '1000', '0000', '0001',
                '0011', '0010', '0110', '0111',
                '0101', '0100', '1100', '1101']
    graycodeidx = [0, 2, 0, 1,
                   0, 3, 0, 1,
                   0, 2, 0, 1,
                   0, 3, 0, 1]
    for i, code in enumerate(graycode):
        if i == 0:
            qc.append(rccx_opt('right'), [address[0], address[1], anc[0]])
            qc.append(rccx_opt(), [address[2], address[3], anc[1]])
            qc.append(rccx_opt(), anc[0:3])
        else:
            if graycodeidx[i-1] <= 1:
                qc.cx(anc[0], anc[2])
                qc.t(anc[2])
                qc.cx(anc[1], anc[2])
                qc.u(pi/2, 0, 3*pi/4, anc[2])
            else:
                qc.cx(anc[1], anc[2])
                qc.t(anc[2])
                qc.cx(anc[0], anc[2])
                qc.u(pi / 2, 0, 3 * pi / 4, anc[2])

        for meteor in problem_set[int(code, 2)]:
            qc.cx(anc[2], data[idx(meteor)])

        if i == 15:
            qc.append(rccx_opt('left'), anc[0:3])
            qc.append(rccx_opt('left'), [address[0], address[1], anc[0]])
            qc.append(rccx_opt('left'), [address[2], address[3], anc[1]])
        else:
            if graycodeidx[i] <= 1:
                qc.u(pi/2, pi/4, pi, anc[2])
                qc.cx(anc[1], anc[2])
                qc.tdg(anc[2])
                qc.cx(anc[0], anc[2])
            else:
                qc.u(pi / 2, pi / 4, pi, anc[2])
                qc.cx(anc[0], anc[2])
                qc.tdg(anc[2])
                qc.cx(anc[1], anc[2])
            if graycodeidx[i] == 0:
                qc.cx(address[1], anc[0])
            elif graycodeidx[i] == 1:
                qc.cx(address[0], anc[0])
            elif graycodeidx[i] == 2:
                qc.cx(address[3], anc[1])
            else:
                qc.cx(address[2], anc[1])

        if i != 15 and graycodeidx[i] != 0:  # last x can be skipped
            qc.x(address[graycodeidx[i]])

    return qc.to_gate(label='QRAM')


"""
    This oracle applies the permanent of the board matrix as a phase to the flag qubit.
"""
def oracle():
    data = QuantumRegister(16)  # data for oracle
    anc = QuantumRegister(7)  # ancillary
    flag = QuantumRegister(1)  # flag register
    qc = QuantumCircuit(data, anc, flag)

    # compute
    qc.append(rccx_opt('right'), [data[0], data[5], anc[0]])
    qc.append(rccx_opt('right'), [data[11], data[14], anc[1]])
    qc.append(rccx_opt('right'), [data[2], data[4], anc[2]])
    qc.append(rccx_opt(), [data[11], data[13], anc[3]])
    qc.append(rccx_opt(), [data[10], data[15], anc[4]])
    qc.append(rccx_opt(), [data[1], data[4], anc[5]])
    qc.append(rccx_opt(), [data[9], data[15], anc[6]])

    # mark solution
    qc.append(double_ccp_opt(), [anc[1], anc[0], anc[4], flag])
    qc.append(double_ccp_opt(), [anc[1], anc[5], anc[4], flag])
    qc.append(double_ccp_opt(), [anc[3], anc[2], anc[6], flag])
    qc.append(rccx_opt('left'), [data[2], data[4], anc[2]])
    qc.append(rccx_opt('right'), [data[0], data[6], anc[2]])
    qc.append(double_ccp_opt(), [anc[3], anc[2], anc[6], flag])

    # uncompute
    qc.append(rccx_opt('left'), [data[10], data[15], anc[4]])
    qc.append(rccx_opt('left'), [data[0], data[6], anc[2]])
    qc.append(rccx_opt('left'), [data[11], data[14], anc[1]])

    # uncompute and compute
    qc.append(double_rccx_opt(), [data[5], data[0], data[7], anc[0]])
    qc.append(double_rccx_opt(), [data[13], data[11], data[12], anc[3]])
    qc.append(double_rccx_opt(), [data[1], data[4], data[3], anc[5]])
    qc.append(double_rccx_opt(), [data[9], data[15], data[8], anc[6]])

    # compute
    qc.append(rccx_opt('right'), [data[10], data[13], anc[1]])
    qc.append(rccx_opt('right'), [data[1], data[6], anc[2]])
    qc.append(rccx_opt('right'), [data[9], data[14], anc[4]])

    # mark solution
    qc.append(double_ccp_opt(), [anc[1], anc[0], anc[4], flag])
    qc.append(double_ccp_opt(), [anc[1], anc[5], anc[4], flag])
    qc.append(double_ccp_opt(), [anc[3], anc[2], anc[6], flag])
    qc.append(rccx_opt('left'), [data[1], data[6], anc[2]])
    qc.append(rccx_opt('right'), [data[2], data[5], anc[2]])
    qc.append(double_ccp_opt(), [anc[3], anc[2], anc[6], flag])

    # uncompute
    qc.append(rccx_opt('left'), [data[0], data[7], anc[0]])
    qc.append(rccx_opt('left'), [data[3], data[4], anc[5]])

    # uncompute and compute
    qc.append(double_rccx_opt(), [data[5], data[2], data[7], anc[2]])
    qc.append(double_rccx_opt(), [data[13], data[10], data[12], anc[1]])
    qc.append(double_rccx_opt(), [data[11], data[12], data[9], anc[3]])
    qc.append(double_rccx_opt(), [data[9], data[14], data[8], anc[4]])
    qc.append(double_rccx_opt(), [data[15], data[8], data[13], anc[6]])

    # compute
    qc.append(rccx_opt('right'), [data[1], data[7], anc[5]])
    qc.append(rccx_opt('right'), [data[3], data[5], anc[0]])

    # mark solution
    qc.append(double_ccp_opt(), [anc[1], anc[0], anc[4], flag])
    qc.append(double_ccp_opt(), [anc[1], anc[5], anc[4], flag])
    qc.append(double_ccp_opt(), [anc[3], anc[2], anc[6], flag])
    qc.append(rccx_opt('left'), [data[2], data[7], anc[2]])
    qc.append(rccx_opt('right'), [data[3], data[6], anc[2]])
    qc.append(double_ccp_opt(), [anc[3], anc[2], anc[6], flag])

    # uncompute
    qc.append(rccx_opt(), [data[3], data[6], anc[2]])
    qc.append(rccx_opt(), [data[8], data[13], anc[6]])
    qc.append(rccx_opt(), [data[1], data[7], anc[5]])
    qc.append(rccx_opt(), [data[8], data[14], anc[4]])
    qc.append(rccx_opt('left'), [data[9], data[12], anc[3]])
    qc.append(rccx_opt('left'), [data[10], data[12], anc[1]])
    qc.append(rccx_opt('left'), [data[3], data[5], anc[0]])

    return qc.to_gate(label='oracle')


"""
    Standard diffusion operator optimized by fusing single quibt gates
"""
def diffusion():
    address = QuantumRegister(4)  # data for oracle
    anc = QuantumRegister(1)
    qc = QuantumCircuit(address, anc)

    qc.u(pi/2, 0, 0, address[0:3])  # --H-X--
    qc.u(pi/2, 0, 0, address[3])  # Technically this is --H-X-H-- but the last H cancels with the first H in the Toffoli gate below
    qc.append(rccx_opt('right'), [address[0], address[1], anc[0]])  # it suffices to use right here because the single qubit gates on the ancillary cancel with the QRAM rccx's
    qc.append(ccx_opt('right'), [address[2], anc[0], address[3]])  # it suffices to use 'right' here because the Hadamard gate cancels
    qc.append(rccx_opt(), [address[0], address[1], anc[0]])
    qc.z(address[3])  # if only one iteration where to be applied this gate can be skipped since any diagonal gate at the end may be ignored
    qc.u(pi/2, pi, pi, address[0:3])  # --X-H--

    return qc.to_gate(label='diffusion')


"""
    Complete circuit composed of
        - Initialization
        - QRAM
        - Oracle
        - QRAM
        - Diffusion
    The circuit uses 4 address qubits, 16 data qubits, a flag qubit and 7 ancillary qubits.
    Its total cost is 6552 using 612 CX and 432 U3 gates.
"""
def week3_ans_func(problem_set):
    address = QuantumRegister(4, 'address')  # address of qram
    data = QuantumRegister(16, 'data')  # data of qram
    anc = QuantumRegister(7, 'anc')  # ancillary qubits
    flag = QuantumRegister(1, 'flag')  # flag qubit
    cr = ClassicalRegister(4, 'classical')
    qc = QuantumCircuit(cr, address, data, anc, flag)

    # initialization
    qc.h(address[:])
    qc.x(flag)  # flag starts in |1>
    niterations = 1
    for _ in range(niterations):
        qc.u(pi/2, pi/4, pi, anc[0])  # saves an additional gate later on in between QRAM^-1 and diffusion
        qc.append(qram(problem_set), address[:] + data[:] + anc[0:3])
        qc.append(oracle(), data[:]+anc[:]+flag[:])
        qc.append(qram(problem_set).inverse(), address[:] + data[:] + anc[0:3])
        qc.append(diffusion(), address[:]+anc[0:1])

    qc.measure(address, cr)
    qc = qc.reverse_bits()

    return qc


In [3]:
# Submission code
from qc_grader.exercises.week3 import prepare_ex3, grade_ex3, submit_ex3, criteria

# Execute your circuit with following prepare_ex3() function.
# The prepare_ex3() function works like the execute() function with only QuantumCircuit as an argument.
job = prepare_ex3(week3_ans_func)

result = job.result()
counts = result.get_counts()
original_problem_set_counts = counts[0]

original_problem_set_counts
# The bit string with the highest number of observations is treated as the solution.

Running week3_ans_func...
Computing cost...
Starting experiments. Please wait...
You may monitor the job (id: 5fcf617f9478000012931520) status and proceed to grading when it successfully completes.


{'0000': 36,
 '0001': 43,
 '0010': 45,
 '0011': 37,
 '0100': 46,
 '0101': 411,
 '0110': 32,
 '0111': 48,
 '1000': 44,
 '1001': 30,
 '1010': 41,
 '1011': 37,
 '1100': 34,
 '1101': 34,
 '1110': 43,
 '1111': 39}

In [4]:
# Check your answer by executing following code.
# The quantum cost of the QuantumCircuit is obtained as the score. The lower the cost, the better.
grade_ex3(job)

Grading your answer. Please wait...

Congratulations 🎉! Your answer is correct.
Your score is 6552.
The lower your score the better!
Feel free to submit your answer.


In [ ]:
# Submit your results by executing following code. You can submit as many times as you like during the period. 
submit_ex3(job)